In [ ]:
!gdown --id '153DyBFeQCCl3-85M0SalJz8z0W_ZKYX5' --output parking.zip # 下载dataset
!unzip parking.zip

In [2]:
import pathlib
import tensorflow as tf

In [6]:
read_file_time = 0

def load_and_preprocess_from_path_label(path, label):
  global read_file_time
  read_file_time += 1
  img = tf.io.read_file(path)
  img = tf.image.decode_jpeg(img, channels=3)
  img = tf.image.resize(img, (128, 128))
  img /= 255.0
  return img, label

In [7]:
label_map = {'free': 0, 'occupy': 1}

train_files_paths = [str(file_path) for file_path in pathlib.Path('./parking/train').glob('*/*')]
train_files_label = [label_map[pathlib.Path(file_path).parent.name] for file_path in train_files_paths]
image_count = len(train_files_paths)

# 构建image, label数据集
ds = tf.data.Dataset.from_tensor_slices((train_files_paths, train_files_label))
image_label_ds = ds.map(load_and_preprocess_from_path_label)

In [ ]:
# 设置一个和数据集大小一致的 shuffle buffer size（随机缓冲区大小）以保证数据
# 被充分打乱。
ds = image_label_ds.shuffle(buffer_size=image_count)
ds = ds.repeat()
ds = ds.batch(batch_size=100)
# 当模型在训练的时候，`prefetch` 使数据集在后台取得 batch。
ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

model = tf.keras.Sequential([
    tf.keras.layers.Convolution2D(24, 3, 3, input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Convolution2D(48, 3, 3),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(ds, epochs=30, steps_per_epoch=300)